In [ ]:
import ndjson

with open("input.ndjson") as file:
    data = ndjson.load(file)

photo_id_to_s3_key = {}
for row in data:
    if row["_type"] == "photo":
        photo_id_to_s3_key[row["_id"]] = row["s3Key"]

data = [row for row in data if row["_type"] != "photo"]

for row in data:
    if row["_type"] == "trip":
        cover_photo = row.pop("coverPhoto", None)
        if cover_photo is not None:
            row["coverPhotoS3Key"] = photo_id_to_s3_key[cover_photo["_ref"]]
        report = row.get("report")
        if report is not None:
            for block in report:
                if block["_type"] == "figure":
                    photo = block.pop("photo", None)
                    if photo is not None:
                        block["photoS3Key"] = photo_id_to_s3_key[photo["_ref"]]
    elif row["_type"] == "landmark":
        photo = row.pop("photo", None)
        if photo is not None:
            row["photoS3Key"] = photo_id_to_s3_key[photo["_ref"]]

with open("output.ndjson", mode="w") as file:
    ndjson.dump(data, file)